In [1]:
import nltk
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ramsay\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Ramsay\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
nltk.download("stopwords")
nltk.download('words')
stop_english = set(stopwords.words('english'))
english_words = set(x.lower() for x in nltk.corpus.words.words())

In [15]:
def clean(df,field):
	print("start cleaning")
	df[field] = df[field].str.replace('[^a-zA-Z ]', '')
	df[field].str.lower()
	df = df.dropna(how='all')
	print("cleaning done")
	return df


def stem_sentences(sentence):
	tokens = sentence.split()
	#Drop stopwords
	stemmed_tokens = [stemmer.stem(token) for token in tokens]
	return ' '.join(stemmed_tokens)

def stem_df(df,field,language): 
	print("Stemming")
	
	global stemmer
	stemmer = SnowballStemmer(language)
	df[field] = df[field].apply(stem_sentences)
	
	return df
    
def drop_stop_df(df,field):
	print("Dropping stop")
	
	df[field] = df[field].apply(drop_stop_sentence)
	
	return df

def drop_stop_sentence(sentence):
	tokens = sentence.split()
	stopped_tokens = [x for x in tokens if x not in stop_english]
	return ' '.join(stopped_tokens)

def remove_english(df,field):
	print("Dropping english")
	df[field] = df[field].apply(drop_eng_sentence)
	
	return df

def drop_eng_sentence(sentence):
	tokens = sentence.split()
	stopped_tokens = [x for x in tokens if x not in english_words]
	return ' '.join(stopped_tokens)

def remove_non_english(df,field):
	print("Dropping english")
	df[field] = df[field].apply(drop_non_eng_sentence)
	
	return df

def drop_non_eng_sentence(sentence):
	tokens = sentence.split()
	stopped_tokens = [x for x in tokens if x in english_words]
	return ' '.join(stopped_tokens)

def apply_tfidf(verbs):
    tfidf = TfidfVectorizer(sublinear_tf= True,
                            min_df= 3,
                            norm= 'l2',
                            encoding= 'latin-1',
                            ngram_range= (1,2),
                            stop_words= 'english',
#                             max_features = 40000,
                           )
    features = tfidf.fit_transform(verbs)
    
    return features, tfidf

In [25]:
def build(features, target, target_name, conf):
            try:
                h2o.cluster().shutdown()
            except:
                pass
            h2o.init(max_mem_size=200)
            print("Train data size : {}, targets size :{}".format(features.shap, target.shape))
            print("Targets : {}".format(target.unique()))
            target = target.astype('category')
            target = target.cat.codes
            obs = target.shape[0]

            if obs >= conf['min_train_records'] and len(target.unique()) > 1:
                feature_list = features.columns
                features[target_name] = target
                
                # build model on training data
                print(features)
                train_data_h = h2o.H2OFrame(features)
                train_data_h[target_name] = train_data_h[target_name].asfactor()
                training_types = (train_data_h.types)

                y = target_name
                x = list(feature_list)
                
                # Partition data into 80% ,20% chunks
                splits = train_data_h.split_frame(ratios=[conf['train_perc']], seed=1)
                train = splits[0]
                valid = splits[1]
                
                print("AutoML Starting ")
                try:
                    aml = H2OAutoML(max_models = int(conf['max_models_fit']),
                                    max_runtime_secs = conf['max_model_runtime'],
                                    seed = 1234,
                                    stopping_metric = conf['stopping_measure'],
                                    balance_classes = True,
                                    nfolds=conf['nfolds'],
                                    include_algos = conf['include_algos'])

                    aml.train(  x = x,
                                y = y,
                                training_frame = train,
                                validation_frame = valid)

                    ###Uncomment below to enable scoring
                    model_path = conf['model_folder']

                    model_name = h2o.save_model(aml.leader,
                                                path = model_path,
                                                force = True)


                    # c_w.write(f'ndicate/model/dtypes_{aml.leader.model_id}.json',training_types,'processing')  ##USED WHERE MODEL HAS SPECIFIC DTYPES PER FEATURE

                    print(f"\n\n\t\t\t\t\t{aml.leader.model_id}\n\n")

                    return True
                except Exception as E:
                    print(f"Failiure Training for {target_name} ")
                    print(E)
                    return False
            else:
                print(f"Not enough training records to build: {target_name}")
                return False

In [7]:
# Format Data
f = open("data/smsspamcollection/SMSSpamCollection", "r")
targets = []
texts = []
for line in f:
    columns = line.split("\t", 1)
    texts.append(columns[1])
    if columns[0] == "ham":
        targets.append(0)
    elif columns[0] == "spam":
        targets.append(1)
    else:
        print("Error")
        break
# for i in range(5):
#     print("{}: {}".format(targets[i], texts[i]))

0: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

0: Ok lar... Joking wif u oni...

1: Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

0: U dun say so early hor... U c already then say...

0: Nah I don't think he goes to usf, he lives around here though



In [13]:
df = pd.DataFrame()
df["target"] = targets
df["text"] = texts
df.head(5)

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
text_field = "text"
text_only = clean(df, text_field)
text_only = remove_non_english(text_only, text_field)
dropped_stop_text = drop_stop_df(text_only, text_field)
stemmed_text = stem_df(dropped_stop_text, text_field, "english")

stemmed_text.head(5)


start cleaning
cleaning done
Dropping english
Dropping stop
Stemming


,target,text
0,0,point crazi n great world la e buffet got wat
1,0,lar u
2,1,entri win final st receiv entri appli
3,0,dun say earli c alreadi say
4,0,dont think goe around though


In [22]:
conf = {
    "text_field": "text",
#     "id_fields" : [BI_ID, audio_id],
	"segment": True,
	"low_memory": False,
    "train_perc": 0.9,
	"min_retrain_cnt": 50,
	"min_train_records": 10, 
    "train_perc": 0.8,
    "model_train_start_date": "1900-01-01",
    "max_models_fit": 5,
    "max_model_runtime": 3600,
    "nfolds": 0,
    "accuracy_measure": "AUC",
    "stopping_measure": "AUC",
    "include_algos": ["GLM", "DRF", "GBM"],
    "exclude_algos": ["StackedEnsemble", "XGBoost", "DeepLearning"],
    "model_folder": "/data/models/"
}

In [27]:
print(f"Starting Training for Spam")
train_df = stemmed_text.loc[~stemmed_text["target"].isna()]
features, tfidf = apply_tfidf(train_df[conf['text_field']])
tfidf_df = pd.DataFrame(features.todense())
if tfidf_df.shape[0] != train_df.shape[0]:
    print("Mis match on training data sizes")
    print("TFIDF Shape {}".format(tfidf_df.shape))
    print("Train Shape {}".format(train_df.shape))
else:
    features_df = pd.merge(tfidf_df, train_df.reset_index(drop=True), left_index=True, right_index=True).copy()  
    features_df.columns = features_df.columns.astype(str)
    training_completed = build(features_df, train_df["target"], "target", conf)
    if training_completed:   
        print("Completed Training")

Starting Training for Spam
H2O session _sid_b1b7 closed.
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from c:\users\ramsay\appdata\local\programs\python\python36\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Ramsay\AppData\Local\Temp\tmpoby2_ba_
  JVM stdout: C:\Users\Ramsay\AppData\Local\Temp\tmpoby2_ba_\h2o_Ramsay_started_from_python.out
  JVM stderr: C:\Users\Ramsay\AppData\Local\Temp\tmpoby2_ba_\h2o_Ramsay_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Africa/Harare
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,10 months and 18 days !!!
H2O cluster name:,H2O_from_python_Ramsay_xsl0me
H2O cluster total nodes:,1
H2O cluster free memory:,200 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Train data size : (5574, 2018), targets size :(5574,)
Targets : [0 1]
        0    1    2    3    4    5    6    7    8    9  ...  2008  2009  2010  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
5     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
6     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
7     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
8     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
9     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
10    0.0  0.0  0.0  0.0  0.0  0.0  0.0

[5574 rows x 2018 columns]
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML Starting 
AutoML progress: |████████████████████████████████████████████████████████| 100%


					GLM_grid_1_AutoML_20200925_131508_model_1


Completed Training


In [191]:
h2o.cluster().shutdown()